In [1]:
pip install pinecone

In [2]:
pip install -U sentence-transformers

#Loadded data


In [3]:
cricket_news = """
The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.
India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.
The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.
"""

football_news = """
The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.
In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.
Both ties promise thrilling encounters, featuring some of the best talents in world football.
"""

election_news = """
As election season heats up, the latest developments reveal a highly competitive atmosphere across several key races.
The presidential election has seen intense campaigning from all major candidates, with recent polls indicating a tight race.
Incumbent President Jane Doe is seeking re-election on a platform of economic stability and healthcare reform, while her main rival, Senator John Smith, focuses on education and climate change initiatives."""


ai_revolution_news = """
The AI revolution continues to transform industries and reshape the global economy.
Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.
Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety."""

In [4]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
embeddings = embedding_model.encode([cricket_news, football_news, election_news, ai_revolution_news])

In [6]:
embeddings

array([[-0.02901838,  0.01924444, -0.01814237, ...,  0.00644329,
        -0.01740803, -0.01381656],
       [-0.00384658, -0.07271527, -0.00284142, ..., -0.02027763,
         0.02123845, -0.03015986],
       [-0.02962374,  0.05711379,  0.01119963, ...,  0.01319242,
         0.02634872,  0.01807422],
       [-0.01667609,  0.05068194, -0.05662726, ..., -0.00878626,
        -0.02318501, -0.04949613]], dtype=float32)

In [9]:
len(embeddings[3])

768

#initialize Pinecone Vectordb

In [10]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key="pcsk_2CS5Dx_8xzAK3W2fUBmi5NGgsYg9kfZ3Hzo1i51JEfXbp5c1GHANfVebFerF8KcAdgjMco")
spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [11]:
pc.create_index("example-index", dimension=768, metric="cosine", spec=spec)

{
    "name": "example-index",
    "metric": "cosine",
    "host": "example-index-r2buutt.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}

In [12]:
pc.list_indexes()

[
    {
        "name": "example-index",
        "metric": "cosine",
        "host": "example-index-r2buutt.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 768,
        "deletion_protection": "disabled",
        "tags": null
    }
]

#index

In [13]:
index = pc.Index("example-index")

In [14]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [15]:
index.upsert([
    {"id":"id1", "values":embeddings[0], "metadata":{'source': 'cricket'}},
    {"id":"id2", "values":embeddings[1], "metadata":{'source': 'football'}},
    {"id":"id3", "values":embeddings[2], "metadata":{'source': 'election'}},
    {"id":"id4", "values":embeddings[3], "metadata":{'source': 'ai_revolution'}}
])

{'upserted_count': 4}

In [16]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4,
 'vector_type': 'dense'}

#Symmetry Search

In [17]:
query = "technology"
query_embedding = embedding_model.encode(query).tolist()

In [18]:
len(query_embedding)

768

In [19]:
similar_docs = index.query(vector=query_embedding, top_k=2, include_metadata=True)
similar_docs

{'matches': [{'id': 'id4',
              'metadata': {'source': 'ai_revolution'},
              'score': 0.218234062,
              'values': []},
             {'id': 'id1',
              'metadata': {'source': 'cricket'},
              'score': 0.0997180939,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')